# Disability Employment Gap

**Author**: Luke Steuber  
**Date**: February 2026

Analysis of disability employment statistics from four sources:
1. Bureau of Labor Statistics (BLS) - 2024 disability employment data
2. Federal Reserve Economic Data (FRED) - 2009-2024 employment trends
3. Equal Employment Opportunity Commission (EEOC) - ADA charge statistics 1992-2024
4. Job Accommodation Network (JAN) - Workplace accommodation costs and benefits

## Key Findings

- **24.5% vs 67% labor force participation gap** (disabled vs non-disabled, 2024)
- **58.7% of workplace accommodations cost $0**
- **Median accommodation cost: $300** (one-time expense)
- **Employment gap narrowing**: From 40pp (2009) to 37.4pp (2024)
- **679,637 ADA charges filed** since 1992 (33 years)

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

%matplotlib inline

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Load Datasets

All JSON files are in the parent directory (`../`).

In [ ]:
# Load BLS data
with open('../bls_disability_employment_2024.json') as f:
    bls_data = json.load(f)

# Load FRED data
with open('../fred_disability_employment.json') as f:
    fred_data = json.load(f)

# Load EEOC data
with open('../eeoc_ada_charges.json') as f:
    eeoc_data = json.load(f)

# Load JAN data
with open('../jan_workplace_accommodations.json') as f:
    jan_data = json.load(f)

print("✓ All datasets loaded successfully")

## 2. BLS Data - 2024 Employment Statistics

In [ ]:
# Extract key statistics
bls_stats = bls_data['overall_statistics']

print("=== BLS 2024 Employment Statistics ===")
print(f"\nWith Disability:")
print(f"  Labor Force Participation Rate: {bls_stats['with_disability']['labor_force_participation_rate']}%")
print(f"  Employment-Population Ratio: {bls_stats['with_disability']['employment_population_ratio']}%")
print(f"  Unemployment Rate: {bls_stats['with_disability']['unemployment_rate']}%")

print(f"\nWithout Disability:")
print(f"  Employment-Population Ratio: {bls_stats['without_disability']['employment_population_ratio']}%")
print(f"  Unemployment Rate: {bls_stats['without_disability']['unemployment_rate']}%")

print(f"\nKey Insights:")
for note in bls_data['overall_statistics']['notes']:
    print(f"  • {note}")

## 3. FRED Data - Historical Trends

In [ ]:
# Convert FRED annual data to DataFrame
fred_annual = fred_data['annual_data']
fred_df = pd.DataFrame([
    {
        'year': int(year),
        'disability_employment_ratio': data.get('disability_employment_ratio'),
        'total_employment_ratio': data.get('total_employment_ratio'),
        'employment_gap_pp': data.get('employment_gap_pp'),
        'disability_unemployment_rate': data.get('disability_unemployment_rate'),
        'total_unemployment_rate': data.get('total_unemployment_rate'),
        'disability_lfpr': data.get('disability_lfpr')
    }
    for year, data in fred_annual.items()
]).sort_values('year')

print(f"FRED data covers {len(fred_df)} years: {fred_df['year'].min()}-{fred_df['year'].max()}")
print(f"\nLatest figures (2024):")
print(fred_df[fred_df['year'] == 2024].T)

## 4. EEOC Data - ADA Charges

In [ ]:
# Convert EEOC annual data to DataFrame
eeoc_df = pd.DataFrame(eeoc_data['annual_data'])

print("=== EEOC ADA Charge Statistics ===")
print(f"\nTotal charges 1992-2024: {eeoc_data['summary']['total_ada_charges_1992_2024']:,}")
print(f"Total monetary benefits: ${eeoc_data['summary']['total_monetary_benefits_millions_usd']:,.1f} million")
print(f"Peak year: {eeoc_data['summary']['peak_year']} ({eeoc_data['summary']['peak_year_charges']:,} charges)")

print(f"\nRecent years:")
print(eeoc_df[eeoc_df['fiscal_year'] >= 2020][['fiscal_year', 'ada_charges_filed', 'monetary_benefits_millions_usd']])

## 5. JAN Data - Accommodation Costs

In [ ]:
# Extract accommodation cost distribution
cost_dist = jan_data['accommodation_costs']['cost_distribution']

print("=== JAN Workplace Accommodation Costs ===")
print(f"\nCost Distribution (n={jan_data['accommodation_costs']['cost_respondents']}):")
for item in cost_dist:
    print(f"  {item['category']}: {item['percent']}% ({item['count']} employers)")
    if 'median_cost_usd' in item:
        print(f"    Median cost: ${item['median_cost_usd']}")
    elif 'median_annual_cost_usd' in item:
        print(f"    Median annual cost: ${item['median_annual_cost_usd']}")

print(f"\nEffectiveness ({jan_data['effectiveness']['respondents']} employers):")
print(f"  Total effective: {jan_data['effectiveness']['total_effective_pct']}%")

print(f"\nTop Direct Benefits:")
for benefit in jan_data['benefits']['direct_benefits'][:3]:
    print(f"  {benefit['benefit']}: {benefit['pct_employers_reporting']}%")

## 6. Visualization: Employment Gap Time Series

FRED data shows disabled vs total population employment-population ratios from 2009-2024.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

# Plot employment ratios
ax.plot(fred_df['year'], fred_df['disability_employment_ratio'], 
        marker='o', linewidth=2.5, label='With Disability', color='#e74c3c')
ax.plot(fred_df['year'], fred_df['total_employment_ratio'], 
        marker='s', linewidth=2.5, label='Total Civilian', color='#3498db')

# Highlight COVID-19 impact
ax.axvspan(2020, 2021, alpha=0.2, color='gray', label='COVID-19 Period')

# Formatting
ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Employment-Population Ratio (%)', fontsize=12, fontweight='bold')
ax.set_title('Disability Employment Gap: 2009-2024', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xticks(fred_df['year'][::2])  # Every other year

# Add annotations for key years
ax.annotate('ADAAA takes effect', xy=(2009, 19.2), xytext=(2009, 25),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=10, ha='center')
ax.annotate('Series high: 22.7%', xy=(2024, 22.7), xytext=(2022, 30),
            arrowprops=dict(arrowstyle='->', color='#e74c3c', lw=1.5),
            fontsize=10, ha='center', color='#e74c3c')

plt.tight_layout()
plt.show()

print(f"Gap narrowed from {fred_df[fred_df['year']==2009]['employment_gap_pp'].values[0]:.1f} pp (2009) to {fred_df[fred_df['year']==2024]['employment_gap_pp'].values[0]:.1f} pp (2024)")

## 7. Visualization: EEOC ADA Charges Per Year

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

# Bar chart of charges
colors = ['#e74c3c' if year == 2016 else '#3498db' for year in eeoc_df['fiscal_year']]
ax.bar(eeoc_df['fiscal_year'], eeoc_df['ada_charges_filed'], color=colors, alpha=0.8)

# Highlight key years
ax.axvline(x=2009, color='green', linestyle='--', linewidth=2, label='ADAAA takes effect (2009)')
ax.axvline(x=2020, color='gray', linestyle='--', linewidth=2, label='COVID-19 pandemic (2020)')

# Formatting
ax.set_xlabel('Fiscal Year', fontsize=12, fontweight='bold')
ax.set_ylabel('ADA Charges Filed', fontsize=12, fontweight='bold')
ax.set_title('EEOC ADA Discrimination Charges: 1992-2024', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, axis='y', alpha=0.3)

# Format y-axis
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))

# Rotate x-axis labels for readability
plt.xticks(eeoc_df['fiscal_year'][::3], rotation=45)

plt.tight_layout()
plt.show()

print(f"Total charges filed: {eeoc_df['ada_charges_filed'].sum():,}")
print(f"Peak year: {eeoc_df.loc[eeoc_df['ada_charges_filed'].idxmax(), 'fiscal_year']} ({eeoc_df['ada_charges_filed'].max():,} charges)")

## 8. Visualization: BLS Employment by Disability Status

In [ ]:
# Prepare comparison data
categories = ['Employment-\nPopulation Ratio', 'Unemployment\nRate', 'Part-time\nWorkers', 'Self-employed']
with_disability = [
    bls_stats['with_disability']['employment_population_ratio'],
    bls_stats['with_disability']['unemployment_rate'],
    bls_stats['with_disability']['part_time_workers_percent'],
    bls_stats['with_disability']['self_employed_percent']
]
without_disability = [
    bls_stats['without_disability']['employment_population_ratio'],
    bls_stats['without_disability']['unemployment_rate'],
    bls_stats['without_disability']['part_time_workers_percent'],
    bls_stats['without_disability']['self_employed_percent']
]

# Create grouped bar chart
fig, ax = plt.subplots(figsize=(12, 7))
x = np.arange(len(categories))
width = 0.35

bars1 = ax.bar(x - width/2, with_disability, width, label='With Disability', color='#e74c3c', alpha=0.85)
bars2 = ax.bar(x + width/2, without_disability, width, label='Without Disability', color='#3498db', alpha=0.85)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Formatting
ax.set_ylabel('Percent', fontsize=12, fontweight='bold')
ax.set_title('BLS 2024 Employment Comparison by Disability Status', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=11)
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Visualization: JAN Accommodation Cost Distribution

In [ ]:
# Extract cost data
cost_categories = [item['category'] for item in cost_dist]
cost_percentages = [item['percent'] for item in cost_dist]
colors_pie = ['#27ae60', '#3498db', '#e67e22']  # Green, blue, orange

# Create pie chart
fig, ax = plt.subplots(figsize=(10, 7))
wedges, texts, autotexts = ax.pie(cost_percentages, labels=cost_categories, autopct='%1.1f%%',
                                    startangle=90, colors=colors_pie,
                                    textprops={'fontsize': 12, 'fontweight': 'bold'},
                                    wedgeprops={'edgecolor': 'white', 'linewidth': 2})

# Make percentage text white for better contrast
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(13)

ax.set_title('JAN Workplace Accommodation Cost Distribution (2019-2024)\n' + 
             f'Based on {jan_data["accommodation_costs"]["cost_respondents"]} employer responses',
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("Key finding: 61% of accommodations cost $0 (policy/schedule changes)")
print("Median one-time expense: $300")
print("Median annual expense: $2,400")

## 10. Summary Statistics Table

In [ ]:
# Create summary DataFrame
summary_data = {
    'Metric': [
        'Labor Force Participation Rate (with disability)',
        'Labor Force Participation Rate (general population)',
        'Participation Gap (percentage points)',
        'Employment Gap (2024, percentage points)',
        'Employment Gap (2009, percentage points)',
        'Unemployment Rate (with disability)',
        'Unemployment Rate (without disability)',
        'Accommodations costing $0',
        'Median one-time accommodation cost',
        'Total ADA charges (1992-2024)',
        'ADA charges (2024)',
        'Accommodation effectiveness ("very" or "extremely")',
    ],
    'Value': [
        '24.5%',
        '~67%',
        '42.5 pp',
        '37.4 pp',
        '40.1 pp',
        '7.5%',
        '4.0%',
        '61% (58.7% cost $0)',
        '$300',
        '679,637',
        '27,457',
        '66%',
    ],
    'Source': [
        'BLS 2024',
        'Estimated (BLS overall - disability)',
        'Calculated',
        'FRED 2024',
        'FRED 2009',
        'BLS 2024',
        'FRED 2024',
        'JAN 2019-2024',
        'JAN 2019-2024',
        'EEOC 1992-2024',
        'EEOC 2024',
        'JAN 2019-2024',
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.index = summary_df.index + 1  # 1-based indexing

print("\n" + "="*80)
print("DISABILITY EMPLOYMENT GAP: KEY FINDINGS")
print("="*80)
print(summary_df.to_string())
print("="*80)

## 11. Key Takeaways

### Employment Gap
- **24.5% vs 67% labor force participation** — a 42.5 percentage point gap between disabled and non-disabled workers
- Gap is **narrowing slowly**: From 40.1pp (2009) to 37.4pp (2024)
- Disabled workers reach **series-high employment** in 2024 (22.7% employment-population ratio)
- Unemployment rate for disabled workers is **nearly double** (7.5% vs 4.0%)

### Workplace Accommodations Are Cheap
- **61% of accommodations cost $0** (schedule/policy changes)
- **Median one-time cost: $300** (screen readers, door openers)
- **88% of employers rate them effective**
- **Top benefits**: Retention (85%), productivity (52%), training cost savings (48%)

### Legal Enforcement
- **679,637 ADA charges** filed since 1992
- **Peak in 2016** (28,073 charges) after ADAAA broadened disability definition
- **Charges rebounding post-COVID**: 27,457 in 2024 (near-record)
- **Top issues**: Termination (49%), reasonable accommodation (38%), harassment (21%)

### Data Sources
1. BLS Current Population Survey (2024 annual averages)
2. FRED Economic Data (annual employment ratios 2009-2024)
3. EEOC ADA Charge Statistics (FY 1992-2024)
4. JAN Employer Survey (2019-2024, n=5,406 employers)